# Building a Postgres DB for Crime Reports

The project demonstrates the process of creating an optimal database named crimes_db to store data on crime reports primarily available in the CSV format.

## 1. Setup

In [4]:
!pip3 install psycopg2-binary

In [5]:
import psycopg2

Connecting to the default postgres db with default postgres user

In [9]:
conn = psycopg2.connect("dbname=postgres user=postgres")
cur = conn.cursor()

Creating a new database crime_db

In [10]:
conn.autocommit = True
cur.execute("CREATE DATABASE crime_db;")
conn.close()

Connecting to the new db and creating the schema

In [12]:
conn = psycopg2.connect("dbname=crime_db user=postgres")
cur = conn.cursor()
cur.execute("CREATE SCHEMA crimes;")

## 2. Tables